In [131]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [121]:
# Load and explore the data
df = pd.read_csv('data/games.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nColumn info:")
print(df.info())
print(f"\nTarget variables stats:")
print(df[['away_score', 'home_score', 'total']].describe())

Dataset shape: (7276, 46)

First few rows:
           game_id  season game_type  week     gameday weekday gametime  \
0  1999_01_MIN_ATL    1999       REG     1  1999-09-12  Sunday      NaN   
1   1999_01_KC_CHI    1999       REG     1  1999-09-12  Sunday      NaN   
2  1999_01_PIT_CLE    1999       REG     1  1999-09-12  Sunday      NaN   
3   1999_01_OAK_GB    1999       REG     1  1999-09-12  Sunday      NaN   
4  1999_01_BUF_IND    1999       REG     1  1999-09-12  Sunday      NaN   

  away_team  away_score home_team  ...  wind  away_qb_id  home_qb_id  \
0       MIN        17.0       ATL  ...   NaN  00-0003761  00-0002876   
1        KC        17.0       CHI  ...  12.0  00-0006300  00-0010560   
2       PIT        43.0       CLE  ...  12.0  00-0015700  00-0004230   
3       OAK        24.0        GB  ...  10.0  00-0005741  00-0005106   
4       BUF        14.0       IND  ...   NaN  00-0005363  00-0010346   

         away_qb_name    home_qb_name          away_coach    home_coach  

In [ ]:
# Data preprocessing and feature engineering
# Define target game (to be excluded from training set to prevent data leakage)
target_game_id = '2025_22_SEA_NE'

# FIRST: Extract target game data BEFORE cleaning/excluding (to reference actual CSV columns)
target_game_data = df[df['game_id'] == target_game_id].copy()

# Remove rows with missing critical values
df_clean = df.dropna(subset=['away_score', 'home_score', 'away_team', 'home_team'])

# SECOND: EXCLUDE TARGET GAME FROM TRAINING DATA (prevent data leakage)
target_game_found = target_game_id in df_clean['game_id'].values
if target_game_found:
    df_clean = df_clean[df_clean['game_id'] != target_game_id]
    print(f"✓ Target game {target_game_id} EXCLUDED from training set (data leakage prevention)")
    print(f"  Preserved original game data for prediction reference")
else:
    print(f"✓ Target game {target_game_id} not in historical data (upcoming game)")
print(f"  Training dataset size: {len(df_clean)} games\n")

# Fill missing values in QB and coach names before using them
df_clean['away_qb_name'] = df_clean['away_qb_name'].fillna('Unknown')
df_clean['home_qb_name'] = df_clean['home_qb_name'].fillna('Unknown')
df_clean['away_coach'] = df_clean['away_coach'].fillna('Unknown')
df_clean['home_coach'] = df_clean['home_coach'].fillna('Unknown')

# Extract features - EXPANDED with QB, coach, game type, weekday, division game, and betting odds
features_to_use = ['season', 'week', 'away_rest', 'home_rest', 'temp', 'wind']
categorical_features = ['away_team', 'home_team', 'roof', 'surface',
                        'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach',
                        'game_type', 'weekday', 'div_game']
numeric_features_new = ['over_odds', 'spread_line']  # Betting lines (pre-game consensus)

# Create a working dataframe
all_features = features_to_use + categorical_features + numeric_features_new
X = df_clean[all_features].copy()
y_away = df_clean['away_score'].copy()
y_home = df_clean['home_score'].copy()
y_total = df_clean['total'].copy()

# Encode categorical variables
le_dict = {}
for cat_feature in categorical_features:
    le = LabelEncoder()
    X[cat_feature] = le.fit_transform(X[cat_feature].astype(str))
    le_dict[cat_feature] = le

# Fill missing values in numeric features
X = X.fillna(X.mean(numeric_only=True))

print(f"Features shape: {X.shape}")
print(f"Features:\n{X.head()}")
print(f"\nAwayScore - Mean: {y_away.mean():.1f}, Std: {y_away.std():.1f}")
print(f"HomeScore - Mean: {y_home.mean():.1f}, Std: {y_home.std():.1f}")
print(f"Total - Mean: {y_total.mean():.1f}, Std: {y_total.std():.1f}")

✓ Target game 2025_20_SF_SEA EXCLUDED from training set (data leakage prevention)
  Preserved original game data for prediction reference
  Training dataset size: 7274 games

Features shape: (7274, 19)
Features:
   season  week  away_rest  home_rest       temp       wind  away_team  \
0    1999     1          7          7  57.926018   8.483282         20   
1    1999     1          7          7  80.000000  12.000000         15   
2    1999     1          7          7  78.000000  12.000000         27   
3    1999     1          7          7  67.000000  10.000000         25   
4    1999     1          7          7  57.926018   8.483282          3   

   home_team  roof  surface  away_qb_name  home_qb_name  away_coach  \
0          1     1        2           252            59          46   
1          5     3        5           121           266          65   
2          7     3        5           195           309          13   
3         11     3        5           255            27    

In [123]:
# Summary of columns used in the model
print("="*60)
print("MODEL INPUT FEATURES FROM GAMES.CSV")
print("="*60)

# Define all used columns
input_features = features_to_use + categorical_features + numeric_features_new
target_columns = ['away_score', 'home_score', 'total']

print(f"\nNUMERIC FEATURES ({len(features_to_use) + len(numeric_features_new)}):")
for feat in features_to_use + numeric_features_new:
    print(f"  - {feat}")

print(f"\nCATEGORICAL FEATURES ({len(categorical_features)}):")
for feat in categorical_features:
    print(f"  - {feat}")

print(f"\nTARGET VARIABLES ({len(target_columns)}):")
for target in target_columns:
    print(f"  - {target}")

print(f"\nTOTAL FEATURES USED: {len(input_features) + len(target_columns)}")

# Show which columns from CSV were NOT used
all_csv_columns = set(df.columns)
used_columns = set(input_features + target_columns)
unused_columns = all_csv_columns - used_columns

if unused_columns:
    print(f"\nCOLUMNS NOT USED IN MODEL ({len(unused_columns)}):")
    for col in sorted(unused_columns):
        print(f"  - {col}")
else:
    print("\nAll columns from CSV were used in the model.")

MODEL INPUT FEATURES FROM GAMES.CSV

NUMERIC FEATURES (8):
  - season
  - week
  - away_rest
  - home_rest
  - temp
  - wind
  - over_odds
  - spread_line

CATEGORICAL FEATURES (11):
  - away_team
  - home_team
  - roof
  - surface
  - away_qb_name
  - home_qb_name
  - away_coach
  - home_coach
  - game_type
  - weekday
  - div_game

TARGET VARIABLES (3):
  - away_score
  - home_score
  - total

TOTAL FEATURES USED: 22

COLUMNS NOT USED IN MODEL (24):
  - away_moneyline
  - away_qb_id
  - away_spread_odds
  - espn
  - ftn
  - game_id
  - gameday
  - gametime
  - gsis
  - home_moneyline
  - home_qb_id
  - home_spread_odds
  - location
  - nfl_detail_id
  - old_game_id
  - overtime
  - pff
  - pfr
  - referee
  - result
  - stadium
  - stadium_id
  - total_line
  - under_odds


## Expanded Model Features

This version includes the high and moderate potential features for improved predictions:

### Features Added (v2):
**High Potential Categorical Features:**
- QB names (away_qb_name, home_qb_name) - QB quality is critical to scoring
- Coaches (away_coach, home_coach) - Coaching philosophy affects scoring patterns
- Game type - Playoff games score differently than regular season
- Weekday - Thursday/Monday night games have different scoring than Sunday
- Division game - Rivalry effects may impact scoring

**Moderate Potential Numeric Features:**
- Betting odds (over_odds, spread_line) - Expert consensus on expected scoring

### Model Improvement:
The expanded feature set (21 total) should improve CV scores by capturing:
- Individual QB impact (not just team identity)
- Coaching philosophy variance
- Game context (playoff vs regular season)
- Expert market expectations

In [124]:
# Train/test split (80/20)
X_train, X_test, y_away_train, y_away_test = train_test_split(X, y_away, test_size=0.2, random_state=42)
_, _, y_home_train, y_home_test = train_test_split(X, y_home, test_size=0.2, random_state=42)
_, _, y_total_train, y_total_test = train_test_split(X, y_total, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 5819
Test set size: 1455


In [125]:
# Train models for away team score
print("=== AWAY TEAM SCORE ===")
rf_away = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_away.fit(X_train, y_away_train)

gb_away = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_away.fit(X_train, y_away_train)

y_away_pred_rf = rf_away.predict(X_test)
y_away_pred_gb = gb_away.predict(X_test)

print(f"Random Forest - MAE: {mean_absolute_error(y_away_test, y_away_pred_rf):.2f}, R²: {r2_score(y_away_test, y_away_pred_rf):.3f}")
print(f"Gradient Boosting - MAE: {mean_absolute_error(y_away_test, y_away_pred_gb):.2f}, R²: {r2_score(y_away_test, y_away_pred_gb):.3f}")

# Train models for home team score
print("\n=== HOME TEAM SCORE ===")
rf_home = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_home.fit(X_train, y_home_train)

gb_home = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_home.fit(X_train, y_home_train)

y_home_pred_rf = rf_home.predict(X_test)
y_home_pred_gb = gb_home.predict(X_test)

print(f"Random Forest - MAE: {mean_absolute_error(y_home_test, y_home_pred_rf):.2f}, R²: {r2_score(y_home_test, y_home_pred_rf):.3f}")
print(f"Gradient Boosting - MAE: {mean_absolute_error(y_home_test, y_home_pred_gb):.2f}, R²: {r2_score(y_home_test, y_home_pred_gb):.3f}")

# Total score is calculated as sum of away + home (mathematical consistency)
print("\n=== TOTAL SCORE ===")
print("Total score is calculated as Away + Home predictions (no separate model)")

=== AWAY TEAM SCORE ===
Random Forest - MAE: 7.95, R²: 0.083
Gradient Boosting - MAE: 7.76, R²: 0.114

=== HOME TEAM SCORE ===
Random Forest - MAE: 8.15, R²: 0.057
Gradient Boosting - MAE: 7.97, R²: 0.099

=== TOTAL SCORE ===
Total score is calculated as Away + Home predictions (no separate model)


In [126]:
# Cross-validation scores (5-fold)
from sklearn.model_selection import cross_val_score

print("="*60)
print("CROSS-VALIDATION SCORES (5-Fold)")
print("="*60)

# Away team score
cv_rf_away = cross_val_score(rf_away, X, y_away, cv=5, scoring='neg_mean_absolute_error')
cv_gb_away = cross_val_score(gb_away, X, y_away, cv=5, scoring='neg_mean_absolute_error')

print("\n=== AWAY TEAM SCORE ===")
print(f"Random Forest - CV MAE: {-cv_rf_away.mean():.2f} (+/- {cv_rf_away.std():.2f})")
print(f"Gradient Boosting - CV MAE: {-cv_gb_away.mean():.2f} (+/- {cv_gb_away.std():.2f})")

# Home team score
cv_rf_home = cross_val_score(rf_home, X, y_home, cv=5, scoring='neg_mean_absolute_error')
cv_gb_home = cross_val_score(gb_home, X, y_home, cv=5, scoring='neg_mean_absolute_error')

print("\n=== HOME TEAM SCORE ===")
print(f"Random Forest - CV MAE: {-cv_rf_home.mean():.2f} (+/- {cv_rf_home.std():.2f})")
print(f"Gradient Boosting - CV MAE: {-cv_gb_home.mean():.2f} (+/- {cv_gb_home.std():.2f})")

# Summary comparison
print("\n" + "="*60)
print("SUMMARY: Cross-Validation Scores")
print("="*60)
print("\n(Total Score calculated as Away + Home, no separate model trained)")
cv_summary = pd.DataFrame({
    'Target': ['Away Score', 'Away Score', 'Home Score', 'Home Score'],
    'Model': ['RF', 'GB', 'RF', 'GB'],
    'CV MAE': [-cv_rf_away.mean(), -cv_gb_away.mean(), -cv_rf_home.mean(), -cv_gb_home.mean()],
    'CV Std': [cv_rf_away.std(), cv_gb_away.std(), cv_rf_home.std(), cv_gb_home.std()]
})
print(cv_summary.to_string(index=False))

CROSS-VALIDATION SCORES (5-Fold)

=== AWAY TEAM SCORE ===
Random Forest - CV MAE: 7.83 (+/- 0.22)
Gradient Boosting - CV MAE: 7.71 (+/- 0.17)

=== HOME TEAM SCORE ===
Random Forest - CV MAE: 7.92 (+/- 0.24)
Gradient Boosting - CV MAE: 7.81 (+/- 0.21)

SUMMARY: Cross-Validation Scores

(Total Score calculated as Away + Home, no separate model trained)
    Target Model   CV MAE   CV Std
Away Score    RF 7.832011 0.217379
Away Score    GB 7.709661 0.173618
Home Score    RF 7.921253 0.237577
Home Score    GB 7.806973 0.207155


## Understanding Cross-Validation Results

**CV MAE (Cross-Validation Mean Absolute Error)** measures average prediction error when the model is tested on data it hasn't seen:
- The number is the expected error in points for score predictions
- The ± value (standard deviation) shows consistency across different data splits
  - Smaller ± = more reliable predictions across different game scenarios
  - Larger ± = predictions vary more depending on which games are used

**Why it matters:** CV scores are more trustworthy than test set scores because they test on multiple different data subsets. A model that performs well on CV will generalize better to future games.

**Interpretation of our results:**
- **Gradient Boosting wins** across all targets (lower MAE = better)
- **Consistency:** Home scores are most consistent (±0.21), suggesting home team conditions are relatively predictable
- **Away score variance:** Higher std (±0.23) suggests away team performance is more variable (travel, crowd, coaching)
- **Practical:** Predictions will typically be within ±7.7-7.9 points of actual scores

In [132]:
# Make prediction for the target game
# (target_game_id already defined in cell 3 during preprocessing)

# Use target_game_data preserved from original df (before training exclusion)
target_game = target_game_data

if not target_game.empty:
    print(f"✓ Found {target_game_id} in original data")
    print(f"\n{'='*60}")
    print("TARGET GAME DATA FROM CSV")
    print(f"{'='*60}")
    # Print all columns and their values
    for col in target_game.columns:
        value = target_game[col].iloc[0]
        print(f"  {col:<25} {value}")
else:
    print(f"Game {target_game_id} not in historical data (upcoming game).")
    print(f"Using trained models with synthesized features from recent games (2023+)...\n")
    # For upcoming games not in dataset, we construct the feature vector
    # Using recent season average values to estimate typical conditions

    # Extract game components
    parts = target_game_id.split('_')
    season = int(parts[0])
    week = int(parts[1])
    away_team = parts[2]
    home_team = parts[3]

    # Get recent data to estimate values
    recent_games = df_clean[df_clean['season'] >= 2023]
    away_avg_rest = recent_games['away_rest'].mean()
    home_avg_rest = recent_games['home_rest'].mean()
    avg_temp = recent_games['temp'].mean()
    avg_wind = recent_games['wind'].mean()

    # Create feature vector for prediction with ALL features including new ones
    # Get average betting odds from recent games
    avg_over_odds = recent_games['over_odds'].mean() if 'over_odds' in recent_games.columns else 45.0
    avg_spread_line = recent_games['spread_line'].mean() if 'spread_line' in recent_games.columns else 0.0

    prediction_data = {
        'season': [season],
        'week': [week],
        'away_rest': [away_avg_rest],
        'home_rest': [home_avg_rest],
        'temp': [avg_temp],
        'wind': [avg_wind],
        'away_team': [away_team],
        'home_team': [home_team],
        'roof': ['outdoors'],  # Default assumption
        'surface': ['grass'],  # Default assumption
        'away_qb_name': ['Unknown'],  # No specific QB data
        'home_qb_name': ['Unknown'],  # No specific QB data
        'away_coach': ['Unknown'],  # No specific coach data
        'home_coach': ['Unknown'],  # No specific coach data
        'game_type': ['REG'],  # Regular season default
        'weekday': ['Sunday'],  # Most common NFL day
        'div_game': ['0'],  # Not a division game by default (as string for categorical encoding)
        'over_odds': [avg_over_odds],  # Average betting odds
        'spread_line': [avg_spread_line]  # Average spread
    }

    prediction_df = pd.DataFrame(prediction_data)

    # Encode categorical features using stored encoders
    for cat_feature in categorical_features:
        if cat_feature in le_dict:
            # Handle unknown values
            known_classes = list(le_dict[cat_feature].classes_)
            value_to_encode = prediction_df[cat_feature].iloc[0]

            # If value is unknown, use the first known class
            if value_to_encode not in le_dict[cat_feature].classes_:
                value_to_encode = known_classes[0]

            prediction_df[cat_feature] = le_dict[cat_feature].transform([value_to_encode])

    print(f"\nPrediction features for {target_game_id}:")
    print(prediction_df)

✓ Found 2025_20_SF_SEA in original data

TARGET GAME DATA FROM CSV
  game_id                   2025_20_SF_SEA
  season                    2025
  game_type                 DIV
  week                      20
  gameday                   2026-01-17
  weekday                   Saturday
  gametime                  20:00
  away_team                 SF
  away_score                6.0
  home_team                 SEA
  home_score                41.0
  location                  Home
  result                    35.0
  total                     47.0
  overtime                  0.0
  old_game_id               2026011701
  gsis                      60171.0
  nfl_detail_id             nan
  pfr                       202601170sea
  pff                       nan
  espn                      401772984
  ftn                       nan
  away_rest                 6
  home_rest                 14
  away_moneyline            245.0
  home_moneyline            -305.0
  spread_line               6.5
  away_spread

In [128]:
# Generate ensemble predictions for the target game
print(f"\n{'='*60}")
print(f"SCORE PREDICTIONS FOR {target_game_id}")
print(f"{'='*60}")

if not target_game.empty:
    # Use the actual game data from dataset
    game_features = target_game[features_to_use + categorical_features + numeric_features_new].copy()

    # Fill NaN values in categorical features
    for cat_feature in categorical_features:
        game_features[cat_feature] = game_features[cat_feature].fillna('Unknown')

    # Encode categorical features
    for cat_feature in categorical_features:
        known_classes = list(le_dict[cat_feature].classes_)
        value_to_encode = game_features[cat_feature].iloc[0]
        if value_to_encode not in le_dict[cat_feature].classes_:
            value_to_encode = known_classes[0]
        game_features[cat_feature] = le_dict[cat_feature].transform([value_to_encode])

    # Fill NaN values in numeric features using training data mean
    game_features = game_features.fillna(X.mean(numeric_only=True))
else:
    game_features = prediction_df

# Get predictions from both models
away_score_rf = rf_away.predict(game_features)[0]
away_score_gb = gb_away.predict(game_features)[0]
away_score_ensemble = (away_score_rf + away_score_gb) / 2

home_score_rf = rf_home.predict(game_features)[0]
home_score_gb = gb_home.predict(game_features)[0]
home_score_ensemble = (home_score_rf + home_score_gb) / 2

# Calculate total as sum of individual scores (mathematical consistency)
total_score_rf = away_score_rf + home_score_rf
total_score_gb = away_score_gb + home_score_gb
total_score_ensemble = away_score_ensemble + home_score_ensemble

# Extract team abbreviations from target_game_id
game_parts = target_game_id.split('_')
away_abbr = game_parts[2]
home_abbr = game_parts[3]

print(f"\n{'ENSEMBLE PREDICTION (Average of RF & GB):':<40}")
print(f"  Away Team ({away_abbr}) Score: {away_score_ensemble:.1f} points")
print(f"  Home Team ({home_abbr}) Score:  {home_score_ensemble:.1f} points")
print(f"  Total Score:           {total_score_ensemble:.1f} points (Away + Home)")


SCORE PREDICTIONS FOR 2025_20_SF_SEA

ENSEMBLE PREDICTION (Average of RF & GB):
  Away Team (SF) Score: 21.0 points
  Home Team (SEA) Score:  28.9 points
  Total Score:           49.9 points (Away + Home)


In [129]:
# Display model comparison
# Extract team abbreviations from target_game_id
game_parts = target_game_id.split('_')
away_abbr = game_parts[2]
home_abbr = game_parts[3]

print(f"\n{'='*60}")
print("MODEL COMPARISON")
print(f"{'='*60}")
print(f"\n{'Metric':<20} {'Random Forest':<20} {'Gradient Boost':<20} {'Ensemble':<15}")
print(f"{'-'*75}")
print(f"{away_abbr + ' Score':<20} {away_score_rf:<20.1f} {away_score_gb:<20.1f} {away_score_ensemble:<15.1f}")
print(f"{home_abbr + ' Score':<20} {home_score_rf:<20.1f} {home_score_gb:<20.1f} {home_score_ensemble:<15.1f}")
print(f"{'Total Score':<20} {total_score_rf:<20.1f} {total_score_gb:<20.1f} {total_score_ensemble:<15.1f}")


MODEL COMPARISON

Metric               Random Forest        Gradient Boost       Ensemble       
---------------------------------------------------------------------------
SF Score             22.3                 19.7                 21.0           
SEA Score            29.0                 28.9                 28.9           
Total Score          51.3                 48.6                 49.9           


In [130]:
# Display feature importance
print(f"\n{'='*60}")
print("FEATURE IMPORTANCE (Random Forest)")
print(f"{'='*60}")

feature_names = features_to_use + numeric_features_new + categorical_features
rf_importance_away = pd.DataFrame({
    'feature': feature_names,
    'importance': rf_away.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nAway Score - Top 10 Features:")
print(rf_importance_away.head(10))

rf_importance_home = pd.DataFrame({
    'feature': feature_names,
    'importance': rf_home.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nHome Score - Top 10 Features:")
print(rf_importance_home.head(10))

print(f"\n(Total Score is calculated as Away + Home, not independently modeled)")


FEATURE IMPORTANCE (Random Forest)

Away Score - Top 10 Features:
         feature  importance
18      div_game    0.150130
10          roof    0.083580
12  away_qb_name    0.080364
11       surface    0.074996
13  home_qb_name    0.070660
4           temp    0.068134
0         season    0.066257
6      over_odds    0.065692
1           week    0.065031
5           wind    0.062155

Home Score - Top 10 Features:
         feature  importance
18      div_game    0.146506
10          roof    0.083052
12  away_qb_name    0.080161
11       surface    0.078673
13  home_qb_name    0.072281
1           week    0.065851
4           temp    0.065674
6      over_odds    0.064996
0         season    0.064187
17       weekday    0.060745

(Total Score is calculated as Away + Home, not independently modeled)
